## Trying out Website Scraping FTW

exploring the interesing field of data mining, initially playing around with some tools to scrape IMDB.

In [2]:
from bs4 import BeautifulSoup
import time
import random
import sys
import urllib.request
import pandas as pd
import json

Lets start with some constants that we will later parametrisize :)

search_url = "https://www.imdb.com/search/title/?release_date=2023-11-01,2024-01-01"

we should be getting start / end date from the user

First lets define some headers as it seems we may be getting blocked without them. Also resort to urllib instead of requests. Let's check if there is any improvement.

In [61]:
search_url = "https://www.imdb.com/search/title/?release_date=2024-01-01,2024-08-01"
root_url = "https://www.imdb.com"
reviews_url = "reviews"
max_reviews = 10

#### Use this to get new content

In [63]:
def get_movie_page(page_url):
    req = urllib.request.Request(page_url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
    req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8')
    req.add_header('Accept-Language', 'en-US,en;q=0.5')
    resp = urllib.request.urlopen(req).read().decode('utf-8')
    return resp
# req = urllib.request.Request(search_url)
# req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
# req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8')
# req.add_header('Accept-Language', 'en-US,en;q=0.5')



### Trying Selenium

for some reason the above function with urllib does not yield the full list of movies. It seems that IMDB have added some async load on some user interaction. I cant find it. Lets try Selenium instead.

> Note: Selenium does not work as it requires installed browser

In [48]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 72.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 18.2 MB/s eta 0:00:00


In [54]:
!pip install pyppeteer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.2
    Uninstalling urllib3-2.2.2:
      Successfully uninstalled urllib3-2.2.2


In [59]:
!pip install nest_asyncio

In [49]:
from selenium import webdriver

def get_movie_page_selenium(page_url):
    # Initialize the WebDriver (you need to download the appropriate WebDriver for your browser)
    driver = webdriver.Firefox()  # Or Chrome(), depending on your preference
    driver.get(page_url)
    
    # Wait for dynamic content to load (you might need to tweak this)
    driver.implicitly_wait(10)  # Waits for up to 10 seconds

    # Get the page source after JavaScript execution
    page_source = driver.page_source
    
    # Close the browser
    driver.quit()
    
    return page_source

In [53]:
html_content = get_movie_page_selenium(search_url)
print(html_content)  # This should include the entire page content, including dynamically loaded movies.

WebDriverException: Message: Process unexpectedly closed with status 255


In [60]:
import asyncio
from pyppeteer import launch
import nest_asyncio
nest_asyncio.apply()


async def get_movie_page_pyppeteer(search_url):
    try:
        # Launch browser
        browser = await launch(headless=True, args=['--no-sandbox', '--disable-gpu'])
        # Open new page
        page = await browser.newPage()
        # Go to the search URL
        await page.goto(search_url)
        # Wait until the page is fully loaded
        await page.waitForSelector('body')
        # Get page content
        content = await page.content()
        # Return the content
        return content
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Ensure that the browser is closed properly
        if browser:
            await browser.close()

# Run the async function and get the page content
page_content = await get_movie_page_pyppeteer(search_url)

# If content is successfully fetched, it will be printed
if page_content:
    print(page_content)


An error occurred: Browser closed unexpectedly:



UnboundLocalError: cannot access local variable 'browser' where it is not associated with a value

### Scraping the List Page to Load Fresh Content

Using this variable we control the date range filter when movie was released and by default movies are sorted by popularity

```search_url = "https://www.imdb.com/search/title/?release_date=2023-09-01,2024-01-01"```

Below is the normal function with urllib `get_movie_page(search_url)`

In [64]:
resp = get_movie_page(search_url)
soup = BeautifulSoup(resp,'html.parser')
movie_blocks = soup.findAll('li',{'class':'ipc-metadata-list-summary-item'})


In [66]:
len(movie_blocks) # it only returns 25 out of 50...

25

In [68]:
movie_blocks[0]

<li class="ipc-metadata-list-summary-item"><div class="ipc-metadata-list-summary-item__c"><div class="ipc-metadata-list-summary-item__tc"><span aria-disabled="false" class="ipc-metadata-list-summary-item__t"></span><div class="sc-74bf520e-3 avhjJ dli-parent"><div class="sc-74bf520e-2 WQsRg"><div class="sc-e5a25b0f-0 iVAGjF dli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div aria-label="add to watchlist" class="ipc-watchlist-ribbon ipc-focusable ipc-watchlist-ribbon--s ipc-watchlist-ribbon--base ipc-watchlist-ribbon--loading ipc-watchlist-ribbon--onImage ipc-poster__watchlist-ribbon" data-testid="poster-watchlist-ribbon-add" role="button" tabindex="0"><svg class="ipc-watchlist-ribbon__bg" height="34px" role="presentation" viewbox="0 0 24 34" width="24px" xmlns="http://www.w3.org/2000/svg"><polygon class="ipc-watchlist-ribbon__bg-ribbon" fill="#000000" points="24 0 0 0 0 32 12.2436611 26.29

Lets save this file so we dont have to scrape it all the time.

In [47]:
with open('./data/imdb_scraped_01-01-01.08.html', 'w') as file:
    file.write(resp)

#### Use this to recover content for testing

In [34]:
# now lets read it and pass it to the soup

with open('./data/imdb_scraped_01-01-01.08.html', 'r') as html:
    soup = BeautifulSoup(html,'html.parser')

In [35]:
movie_blocks = soup.findAll('li',{'class':'ipc-metadata-list-summary-item'})
# test_name = movie_blocks[10].find('h3', {'class': 'ipc-title__text'}).get_text()
# link = movie_blocks[10].find('a', {'class': 'ipc-title-link-wrapper'}, href=True)
# stars = movie_blocks[10].find('span', {'class': 'ipc-rating-star'}).contents[1]
# movie_blocks[10].find('span', {'class': 'metacritic-score-box'}).get_text()
# test_name, stars, root_url + link['href'].split('?')[0] + reviews_url


In [36]:
len(movie_blocks)

25

#### Recording the search result into a dataframe for view

The below code would parse and record our search results into a usefull dataframe

In [19]:
def get_movie_content(movie_snippet):
    title = movie_snippet.find('h3', {'class': 'ipc-title__text'}).get_text()
    title = " ".join(title.split()[1:])
    stars = float(movie_snippet.find('span', {'class': 'ipc-rating-star--rating'}).contents[0])
    url = movie_snippet.find('a', {'class': 'ipc-title-link-wrapper'}, href=True)
    link = root_url + url["href"].split('?')[0] + reviews_url
    try:
        votes = movie_snippet.find('span', {'class': 'ipc-rating-star--voteCount'}).contents[2]
    except:
        votes = 0
    try:
        metascore = int(movie_snippet.find('span', {'class': 'metacritic-score-box'}).get_text())
    except:
        metascore = 'N/A'
    try:
        type = movie_snippet.find('span', {'class': 'dli-title-type-data'}).get_text()
    except:
        type = "Movie"
    return {
        "title": title,
        "stars": stars,
        "link": link,
        "meta": metascore,
        "votes": votes,
        "type": type
    }

In [69]:
movies_list = []
for movie_snippet in movie_blocks:
    movies_list.append(get_movie_content(movie_snippet))

df = pd.DataFrame(movies_list)
df_sorted = df.sort_values(by=["stars"], ascending=False).reset_index(drop=True)
df_sorted

,title,stars,link,meta,votes,type
0,Fallout,8.4,https://www.imdb.com/title/tt12637874/reviews,N/A,243K,TV Series
1,Deadpool & Wolverine,8.0,https://www.imdb.com/title/tt6263850/reviews,56,226K,Movie
2,Inside Out 2,7.8,https://www.imdb.com/title/tt22022452/reviews,73,100K,Movie
3,Presumed Innocent,7.7,https://www.imdb.com/title/tt17677860/reviews,N/A,40K,TV Series
4,Furiosa: A Mad Max Saga,7.6,https://www.imdb.com/title/tt12037194/reviews,79,162K,Movie
5,Batman: Caped Crusader,7.3,https://www.imdb.com/title/tt14681596/reviews,N/A,9.1K,TV Series
6,The Outrun,7.2,https://www.imdb.com/title/tt11687002/reviews,70,357,Movie
7,Longlegs,7.1,https://www.imdb.com/title/tt23468450/reviews,77,48K,Movie
8,Kingdom of the Planet of the Apes,7.0,https://www.imdb.com/title/tt11389872/reviews,66,105K,Movie
9,Twisters,6.8,https://www.imdb.com/title/tt12584954/reviews,65,68K,Movie


#### Try and get the reviews

Now that we have some list to work with, lets try and get the reviews for a random movie

We then save it to a file for testing. Should extract this as a function and cycle the movies from the movie_blocks with some delay. The function should return the scraped and souped page.

Then run another function in order to scrape the reviews and return them as a list of strings without HTML tags and concatenated title + review.

We should combine this in the main movie_blocks list and save it as json with proper date range in the file name.

In [21]:
rand_review = random.randint(0,len(df))
resp = get_movie_page(df_sorted.iloc[rand_review]["link"])
with open('./data/imdb_scraped_movie.html', 'w') as file:
    file.write(resp)

In [22]:
with open('./data/imdb_scraped_movie.html', 'r') as html:
    soup = BeautifulSoup(html,'html.parser')

In [23]:
review_blocks = soup.find_all('div', {'class': 'lister-item'})

In [24]:
def get_movie_review(review_block): 
    review_copy = review_block.find('div', {'class': 'text show-more__control'}).get_text(separator=" ", strip=True)
    review_title = review_block.find('a', {'class': 'title'}).get_text(strip=True)
    review_date = review_block.find('span', {'class': 'review-date'}).get_text()
    try: 
        review_stars = int(review_block.select_one('.rating-other-user-rating span').get_text())
    except:
        review_stars = 0
    return {
        "copy": review_copy,
        "title": review_title,
        "date": review_date,
        "stars": review_stars
    }

In [25]:
reviews = []
for block in review_blocks:
    reviews.append(get_movie_review(block))

In [26]:
df_reviews = pd.DataFrame(reviews)
df_reviews

,copy,title,date,stars
0,Longlegs is an interesting film. It's sometime...,The marketing was a blessing and a curse,12 July 2024,7
1,His legs were literally normal length Honestly...,"bro, false advertising",12 July 2024,7
2,Long Legs is yet another movie that underperfo...,"Had so much potential, let me explain",12 July 2024,6
3,I saw the film for the second time last night ...,The hype might hurt it.,9 July 2024,0
4,With the bar for modern horror movies being so...,Exposition Dump Ending :(,13 July 2024,6
5,"This movie is very good. Atmospheric, beautifu...",A victim of its own hype,28 July 2024,8
6,To say I was excited for this movie was an und...,I REALLY wanted to like it more (6.5/10),15 July 2024,6
7,Cage side quests of dedicating himself to any ...,Cage continues his legendary versatile renaiss...,10 July 2024,8
8,It is hard to think of a better promotion stra...,Turns It Up To Eleven,26 July 2024,7
9,"I was way too hyped for this, obviously the co...",lower your expectations,13 July 2024,6


#### Extract a function that returns reviews

It takes an HTML response. It can be from a file or from IMDB get page function

In [28]:
# this should work with getting the HTML response from IMDB or saved as file.
def get_movie_reviews(html_resp):
    soup = BeautifulSoup(html_resp,'html.parser')
    review_blocks = soup.find_all('div', {'class': 'lister-item'})[:max_reviews] # lets limit to top 10 for now.
    reviews = []
    for block in review_blocks:
        reviews.append(get_movie_review(block))    
    return reviews

#### Try and Pull 3 Movies with Delay

In [30]:
movies_with_reviews = []
for block in [movie_blocks[10]]:
    movie = get_movie_content(block)
    resp = get_movie_page(movie["link"])
    movie["reviews"] = get_movie_reviews(resp)
    movies_with_reviews.append(movie)
    time.sleep(random.randint(10, 20))

movies_json = {"movies": movies_with_reviews}
# with open('./data/imdb_scraped_reviews_06.01-01.15.json', 'w') as f:
#     json.dump(movies_json, f)

In [31]:
len(movies_with_reviews), len(movie_blocks)

(1, 25)

In [33]:
pd.DataFrame(movies_with_reviews[0])

,title,stars,link,meta,votes,type,reviews
0,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'I gave it a7 measured against everyt...
1,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'I think many have a lack of understa...
2,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'It's a very watchable show. Not grea...
3,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'It is good. Looks great. Has great a...
4,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'I'm so happy I saw the positive revi...
5,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'Just a fun show to watch in my opini...
6,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'This series has nothing to do with t...
7,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'My goodness but doesn't anyone want ...
8,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'Just finished watching all the episo...
9,The Decameron,6.3,https://www.imdb.com/title/tt21831910/reviews,N/A,5.3K,TV Series,{'copy': 'Well as I said its beautifully shot ...


In [59]:
movies_with_reviews[24]["title"]

'Chicken Run: Dawn of the Nugget'

In [123]:
movies_json = {"movies": movies_with_reviews}
with open('./data/imdb_scraped_3_movies.json', 'w') as f:
    json.dump(movies_json, f)